Este archivo contiene un pequeño ETL realizado previo a subir la data hacia el bigquery.

**IMPORTACION DE LIBRERIAS**

In [1]:
import numpy as np
import pandas as pd
import ast

**CARGAMOS METADATA**

In [2]:
metadata = pd.read_csv('./data/transform/google/metadata/metadata.csv')

**ELIMINAMOS DUPLICADOS EN GMAP_ID**

In [3]:
print('Shape antes:',  metadata.shape)
metadata.drop_duplicates(subset='gmap_id', inplace=True)
print('Shape despues:',  metadata.shape)

Shape antes: (3025011, 15)
Shape despues: (2998428, 15)


In [4]:
metadata.describe()

,latitude,longitude,avg_rating,num_of_reviews
count,2.998428e+06,2.998428e+06,2.998428e+06,2.998428e+06
mean,3.742927e+01,-9.234204e+01,4.303666e+00,4.384394e+01
std,5.425494e+00,1.635375e+01,7.119273e-01,1.435629e+02
min,-4.092931e+01,-1.788071e+02,1.000000e+00,1.000000e+00
25%,3.370949e+01,-9.925989e+01,4.000000e+00,7.000000e+00
50%,3.827702e+01,-8.759823e+01,4.500000e+00,1.800000e+01
75%,4.129133e+01,-8.029603e+01,4.800000e+00,4.400000e+01
max,8.785652e+01,1.800000e+02,5.000000e+00,9.998000e+03


<hr>

**CATEGORY**

"CATEGORY" es una columna que contiene las categorias en lista a la que pertenece cada business.

Aca vamos a pasar de [x, y, z] a x|y|z ya que string es mas facil de trabajar y es mas flexible.

In [7]:
def from_list_to_str(v):
    if isinstance(v, float):
        return v
    try:
        v = ast.literal_eval(v)
    except:
        print('No se pudo convertir a lista:', v)
        return v
    return '|'.join([category.lower().strip() for category in v])

In [8]:
metadata['new_category'] = metadata.category.apply(from_list_to_str)

In [7]:
metadata.columns

Index(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC',
       'state', 'relative_results', 'url', 'new_category'],
      dtype='object')

<hr>

**HOURS**

"HOURS" es una columna que contiene las horas por dia en las que el negocio abre, de lunes a domingo.

El formato es el siguiente: [dias] = [[dia1: horario1], [dia2: horario2], ... ]]

In [9]:
from calendar import day_name

day_name = [day.lower() for day in day_name]

def week_schedule(v):
    # Si es NA lo devolvemos.
    if isinstance(v, float):
        return v

    # Caso contrario, creamos schedule que es un diccionario que mapea dia: horario
    schedule = {}
    for info in ast.literal_eval(v):
        try:
            schedule[info[0].lower()] = info[1]
        except:
            print(v)
            return v
    return schedule

In [10]:
metadata.hours[10000]

"[['Wednesday', '9AM–5PM'], ['Thursday', '9AM–5PM'], ['Friday', '9AM–5PM'], ['Saturday', 'Closed'], ['Sunday', 'Closed'], ['Monday', '9AM–5PM'], ['Tuesday', '9AM–5PM']]"

En metadata, vamos a crear una columna para cada dia. Asi, metadata contendra 7 columnas, cuyo valores seran los horarios.

In [11]:
schedule = {day:[] for day in day_name}
for v in metadata.hours:
    # Obtenemos los horarios para la institucion, es decir {dia1: horario1, dia2:horario2, etc...}
    output = week_schedule(v)
    for day in day_name:
        schedule[day].append(output[day] if type(output) == dict else output)

In [13]:
# Creamos las columnas
for key, value in schedule.items():
    metadata[key + '_schedule'] = value

In [12]:
metadata.shape

(2998428, 23)

**MISC**

MISC es un diccionario que contiene distintas actividades realizadas por el business o caracteristicas como accesibilidad, etc.

In [14]:
rasgos = set()
for v in metadata.MISC:
    if type(v) == float:
        continue
    else:
        rasgos.update(ast.literal_eval(v).keys())

In [16]:
'|'.join([x.lower() for x in ast.literal_eval(metadata.MISC[0])['Service options']])

'in-store shopping|same-day delivery'

In [17]:
misc_null = metadata.MISC.isnull()
misc_notnull = False == misc_null

In [20]:
# MISC es un diccionario de la forma key (actividad|caracteristicas) = value (lista para todos los business)
misc = {key:[] for key in rasgos}

for v in metadata.MISC:
    # Si es null el campo, entonces para este business todas las actividades van a ser nan
    if type(v) == float:
        for key in misc.keys():
            misc[key].append(np.nan)
    # De lo contrario, convertimos v a diccionario (esta en formato string)
    else:
        data = ast.literal_eval(v)
        # Almacenamos el valor correspondiente en la actividad correspondiente
        for key in misc.keys():
            try:
                value = data[key]
                # En caso de que tenga mas de un valor, los separamos por | como el ejemplo de arriba.
                value = '|'.join([x.lower() for x in value])
            except:
                value = np.nan
            finally:
                misc[key].append(value)

In [21]:
# Creamos las columnas para cada actividad
for key in misc:
    metadata[key] = misc[key]

In [22]:
# Convertimos el nombre de todas las columnas a lowercase
metadata.columns = metadata.columns.str.lower()

In [23]:
# Creamos una nueva columna id, ya que gmap_id es muy poco practica e intuitiva.
metadata.loc[:, 'company_id'] = np.arange(1, metadata.shape[0] + 1)

In [24]:
metadata.columns

Index(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'misc',
       'state', 'relative_results', 'url', 'new_category', 'monday_schedule',
       'tuesday_schedule', 'wednesday_schedule', 'thursday_schedule',
       'friday_schedule', 'saturday_schedule', 'sunday_schedule',
       'popular for', 'lodging options', 'payments', 'planning',
       'dining options', 'getting here', 'recycling', 'amenities',
       'health and safety', 'highlights', 'service options', 'offerings',
       'atmosphere', 'health & safety', 'crowd', 'activities',
       'from the business', 'accessibility', 'company_id'],
      dtype='object')

In [25]:
# Almacenamos el nuevo dataframe
metadata.to_csv('./data/etl/google/metadata.csv', header=True, mode='w', index=False, sep=',')